# PyMOL Remote demo

### 1. [Prerequisite] Server side (where PyMOL is running)

This demo should be run on the `client` side, i.e. the machine where you want to run your Python code.
To run it, you will need to have run the following command on the `server` side, i.e. the machine where PyMOL is running:

1. **Server side (where PyMOL is running)**
```bash
# Start PyMOL with the RPC server as explained in 2.1.
# By default, it will listen on localhost:9123
pymol_remote
```

2. **Client side (on your remote machine)**
Connect to the machine (e.g. a cloud server, your HPC, etc.) where you want to run your Python code via SSH and set up port forwarding.
You will need to use the same port number as the one you used when starting PyMOL on the server side, by default 9123.
```bash
# Set up SSH port forwarding (run this in a terminal)
ssh -R 9123:localhost:9123 username@server_address
```

For more details on how to use `pymol_remote`, see the official usage instructions on the GitHub page or [Martin Buttenschön's short blog post](https://www.blopig.com/blog/2024/11/controlling-pymol-from-afar/) on controlling PyMOL from afar.

### 2. Client side (where you want to run your Python code)

In [1]:
# Option 1: Manually specify the hostname and port -- for maximum flexibility
# NOTE: When you run `pymol_remote` on the server side, it will print a likely guess of your
#  IP address (and the correct port number, 9123 by default).
# from pymol_remote.client import PymolSession
# HOSTNAME = "localhost"
# PORT = 9123
# pymol = PymolSession(hostname=HOSTNAME, port=PORT)

# Option 2: Use the default settings -- for convenience
# NOTE: Even simpler, if you use the default settings, you can just run:
from pymol_remote import get_pymol_session

pymol = get_pymol_session()

# You can now send commands to PyMOL
pymol.fetch("6lyz")
pymol.do("remove solvent")
pymol.do("set valence, on")
pymol.get_state(format="cif");

2025-02-16 11:28:52,938 - pymol-remote:client - INFO - Connecting to PyMol RPC server at `localhost:9123`
2025-02-16 11:28:52,949 - pymol-remote:client - INFO - Connecting to PyMol RPC server at `localhost:9124`


In [2]:
# Make pymol look pretty (Courtesy of Kate Fie: https://www.blopig.com/blog/2024/12/making-pretty-pictures-in-pymol-v2/)
from pymol_remote.style import make_pymol_pretty

make_pymol_pretty(pymol)

Find a specific command

In [3]:
pymol.find_command("name")

['rename',
 'set_name',
 'get_names',
 'count_frames',
 'get_legal_name',
 'get_unused_name',
 'multifilenamegen',
 'get_names_of_type',
 'scene_recall_message',
 'get_drag_object_name',
 'filename_to_objectname']

Get help on a specific command

In [4]:
pymol.help("get_names")

get_names(type='public_objects', enabled_only=0, selection='', *, _self=<module 'pymol.cmd' from '/Applications/PyMOL.app/Contents/lib/python3.11/site-packages/pymol/cmd.py'>)

DESCRIPTION

    "get_names" returns a list of object and/or selection names.

PYMOL API

    cmd.get_names( [string: "objects"|"selections"|"all"|"public_objects"|"public_selections"] )

NOTES

    The default behavior is to return only object names.

SEE ALSO

    get_type, count_atoms, count_states
        


Show all the available commands

In [5]:
pymol.help()

Get help for a specific command by passing the command name to the `help` method.
For example:
```
session.help('fetch')
```

To get links to more documentation, call `session.print_help()`.
Available commands:

  -get_cifstr
  -set_geometry
  -sculpt_activate
  -map_sc
  -set_scene_message
  -undo_copy_sele
  -label
  -fb_module_sc
  -load
  -undo_restore_mode
  -get_setting_float
  -pseudoatom
  -rename
  -callout
  -count_frames
  -get_color_tuple
  -alignto
  -transform_selection
  -count_states
  -ungroup
  -uniquify
  -desaturate
  -get_wizard_stack
  -pi_interactions
  -get_coordset
  -loadable
  -is_alive
  -window_sc
  -get_selection_state
  -mmove
  -madd
  -get_movie_locked
  -space_sc
  -overlap
  -copy
  -slice_new
  -mappend
  -mplay
  -get_pdbstr
  -show_help
  -log
  -get_area
  -get_version
  -refresh
  -label2
  -safe_list_eval
  -volume_ramp_new
  -get_vrml
  -commands
  -get_editor_scheme
  -resume
  -get_symmetry
  -get_legal_name
  -torsion
  -h_fill
  -location_s

### Transfer a structure from PyMOL to Biotite

In [6]:
from biotite.database import rcsb
from biotite.structure.io import pdbx
import warnings

warnings.filterwarnings("ignore")  # ... ignore biotite warnings for now

data = rcsb.fetch("6lyz", "cif")

# Show the first 3 atoms of the structure
pdbx.get_structure(pdbx.CIFFile.read(data), model=1)[:3]

array([
	Atom(np.array([ 3.287, 10.092, 10.329], dtype=float32), chain_id="A", res_id=1, ins_code="", res_name="LYS", hetero=False, atom_name="N", element="N"),
	Atom(np.array([ 2.445, 10.457,  9.182], dtype=float32), chain_id="A", res_id=1, ins_code="", res_name="LYS", hetero=False, atom_name="CA", element="C"),
	Atom(np.array([ 2.5  , 11.978,  9.038], dtype=float32), chain_id="A", res_id=1, ins_code="", res_name="LYS", hetero=False, atom_name="C", element="C")
])

In [7]:
import io

data_from_pymol = pymol.get_state(format="cif")

# Show the first 3 atoms of the structure
pdbx.get_structure(pdbx.CIFFile.read(io.StringIO(data_from_pymol)), model=1)[:3]

array([
	Atom(np.array([ 3.287, 10.092, 10.329], dtype=float32), chain_id="A", res_id=1, ins_code="", res_name="LYS", hetero=False, atom_name="N", element="N"),
	Atom(np.array([ 2.445, 10.457,  9.182], dtype=float32), chain_id="A", res_id=1, ins_code="", res_name="LYS", hetero=False, atom_name="CA", element="C"),
	Atom(np.array([ 2.5  , 11.978,  9.038], dtype=float32), chain_id="A", res_id=1, ins_code="", res_name="LYS", hetero=False, atom_name="C", element="C")
])

In [8]:
### Transfer a structure from Biotite to PyMOL
data = rcsb.fetch("5ocm", "cif")

structure = pdbx.get_assembly(pdbx.CIFFile.read(data), assembly_id="1", model=1)

structure_cif = pdbx.CIFFile()
pdbx.set_structure(structure_cif, structure, include_bonds=True)

buffer = io.StringIO()
structure_cif.write(buffer)

pymol.set_state(buffer.getvalue(), object="5ocm_from_biotite", format="cif")

### Clean up

In [9]:
pymol.delete("all")